# RustLab I/O Module Showcase

Demonstrating the clean, math-first I/O API for reading and writing matrices and vectors.

## Key Features
- **Just 2 core functions**: `save()` and `load()` (with precision and skip variants)
- **Automatic format detection** based on file extensions
- **Smart header detection** - automatically skips non-numeric headers
- **Scientific notation** for very large/small numbers
- **Works with both** ArrayF64 and VectorF64

In [2]:
:dep rustlab-math = { path = ".." }

In [3]:
// Import the clean I/O API
use rustlab_math::{ArrayF64, VectorF64, array64, vec64};
use rustlab_math::io::MathIO;
use std::fs;

## 1. Basic Matrix I/O

Create and save a matrix, then load it back.

In [4]:
// Create a sample matrix
let matrix = array64![
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
];

println!("Original matrix shape: {:?}", matrix.shape());
println!("Original matrix:\n{:?}", matrix);

Original matrix shape: (3, 3)
Original matrix:
Array { inner: [
[1.0, 2.0, 3.0],
[4.0, 5.0, 6.0],


In [5]:
// Save to CSV (auto-detected by .csv extension)
matrix.save("../data/demo_matrix.csv").unwrap();
println!("✓ Matrix saved to demo_matrix.csv");

// View the saved file
let content = fs::read_to_string("../data/demo_matrix.csv").unwrap();
println!("\nFile content:");
println!("{}", content);

[7.0, 8.0, 9.0],
] }
✓ Matrix saved to demo_matrix.csv

File content:
1.000000,2.000000,3.000000


In [6]:
// Load it back
let loaded = ArrayF64::load("../data/demo_matrix.csv").unwrap();
println!("Loaded matrix shape: {:?}", loaded.shape());
println!("Loaded matrix:\n{:?}", loaded);

// Verify they're identical
let original_vec = matrix.to_vec();
let loaded_vec = loaded.to_vec();
let identical = original_vec.iter().zip(loaded_vec.iter()).all(|(a, b)| (a - b).abs() < 1e-15);
println!("\n✓ Matrices are identical: {}", identical);

4.000000,5.000000,6.000000
7.000000,8.000000,9.000000

Loaded matrix shape: (3, 3)
Loaded matrix:
Array { inner: [
[1.0, 2.0, 3.0],
[4.0, 5.0, 6.0],
[7.0, 8.0, 9.0],


## 2. Vector I/O

Vectors are saved as column vectors in text files, row vectors in CSV files.

In [7]:
// Create a sample vector
let vector = vec64![10.0, 20.0, 30.0, 40.0, 50.0];
println!("Original vector: {:?}", vector);

// Save as text file (column format)
vector.save("../data/demo_vector.txt").unwrap();
println!("\n✓ Vector saved to demo_vector.txt");

let content = fs::read_to_string("../data/demo_vector.txt").unwrap();
println!("Text file content (column format):");
println!("{}", content);

] }

✓ Matrices are identical: true
Original vector: Vector { inner: [10.0, 20.0, 30.0, 40.0, 50.0] }

✓ Vector saved to demo_vector.txt
Text file content (column format):
10.000000
20.000000
30.000000
40.000000


In [8]:
// Save as CSV file (row format)
vector.save("../data/demo_vector.csv").unwrap();
println!("✓ Vector saved to demo_vector.csv");

let content = fs::read_to_string("../data/demo_vector.csv").unwrap();
println!("CSV file content (row format):");
println!("{}", content);

50.000000

✓ Vector saved to demo_vector.csv
CSV file content (row format):
10.000000,20.000000,30.000000,40.000000,50.000000



In [9]:
// Load vector back
let loaded_vector = VectorF64::load("../data/demo_vector.txt").unwrap();
println!("Loaded vector: {:?}", loaded_vector);

// Verify identical
let identical = vector.to_slice().iter().zip(loaded_vector.to_slice().iter())
    .all(|(a, b)| (a - b).abs() < 1e-15);
println!("✓ Vectors are identical: {}", identical);

Loaded vector: Vector { inner: [10.0, 20.0, 30.0, 40.0, 50.0] }
✓ Vectors are identical: true


## 3. High Precision I/O

Use `save_with_precision()` for high-precision scientific data.

In [10]:
// Create matrix with high precision values
let pi_matrix = array64![
    [std::f64::consts::PI, std::f64::consts::E],
    [std::f64::consts::TAU, std::f64::consts::SQRT_2]
];

println!("High precision matrix:");
for i in 0..2 {
    for j in 0..2 {
        print!("{:.15} ", pi_matrix.get(i, j).unwrap());
    }
    println!();
}

High precision matrix:
3.141592653589793 2.718281828459045 
6.283185307179586 1.414213562373095 


()

In [11]:
// Save with default precision (6 digits)
pi_matrix.save("../data/pi_default.csv").unwrap();
let default_content = fs::read_to_string("../data/pi_default.csv").unwrap();
println!("Default precision (6 digits):");
println!("{}", default_content);

// Save with high precision (15 digits)
pi_matrix.save_with_precision("../data/pi_high.csv", 15).unwrap();
let high_content = fs::read_to_string("../data/pi_high.csv").unwrap();
println!("High precision (15 digits):");
println!("{}", high_content);

Default precision (6 digits):
3.141593,2.718282
6.283185,1.414214

High precision (15 digits):
3.141592653589793,2.718281828459045
6.283185307179586,1.414213562373095



In [12]:
// Compare precision loss
let default_loaded = ArrayF64::load("../data/pi_default.csv").unwrap();
let high_loaded = ArrayF64::load("../data/pi_high.csv").unwrap();

println!("Original π: {:.15}", std::f64::consts::PI);
println!("Default precision: {:.15}", default_loaded.get(0, 0).unwrap());
println!("High precision: {:.15}", high_loaded.get(0, 0).unwrap());

let error_default = (default_loaded.get(0, 0).unwrap() - std::f64::consts::PI).abs();
let error_high = (high_loaded.get(0, 0).unwrap() - std::f64::consts::PI).abs();
println!("\nError with default precision: {:.2e}", error_default);
println!("Error with high precision: {:.2e}", error_high);

Original π: 3.141592653589793
Default precision: 3.141593000000000
High precision: 3.141592653589793



## 4. Scientific Notation

The I/O module automatically uses scientific notation for very large or very small numbers.

In [13]:
// Create matrix with extreme values
let extreme_matrix = array64![
    [1.23e-8, 4.56e12],     // Very small and very large
    [0.00001, 1000000.0],   // Normal range  
    [9.87e-15, 3.21e20]     // Extremely small and large
];

extreme_matrix.save("../data/extreme_values.csv").unwrap();
let content = fs::read_to_string("../data/extreme_values.csv").unwrap();
println!("Scientific notation automatically applied:");
println!("{}", content);

Error with default precision: 3.46e-7
Error with high precision: 0.00e0
Scientific notation automatically applied:
1.230000e-8,4.560000e12
1.000000e-5,1000000.000000


In [14]:
// Load back and verify precision
let loaded_extreme = ArrayF64::load("../data/extreme_values.csv").unwrap();
println!("Loaded extreme values:");
for i in 0..3 {
    for j in 0..2 {
        let original = extreme_matrix.get(i, j).unwrap();
        let loaded = loaded_extreme.get(i, j).unwrap();
        let relative_error = if original != 0.0 {
            ((loaded - original) / original).abs()
        } else {
            (loaded - original).abs()
        };
        println!("  [{},{}]: {:.3e} -> {:.3e} (rel. error: {:.2e})", 
                 i, j, original, loaded, relative_error);
    }
}

9.870000e-15,3.210000e20

Loaded extreme values:
  [0,0]: 1.230e-8 -> 1.230e-8 (rel. error: 0.00e0)
  [0,1]: 4.560e12 -> 4.560e12 (rel. error: 0.00e0)
  [1,0]: 1.000e-5 -> 1.000e-5 (rel. error: 0.00e0)
  [1,1]: 1.000e6 -> 1.000e6 (rel. error: 0.00e0)
  [2,0]: 9.870e-15 -> 9.870e-15 (rel. error: 0.00e0)
  [2,1]: 3.210e20 -> 3.210e20 (rel. error: 0.00e0)


()

## 5. Skip Rows Functionality

Use `load_skip()` to skip metadata lines at the beginning of files.

In [15]:
// Create a file with metadata header
let metadata_content = r#"# Experimental Data
# Date: 2024-01-01
# Instrument: Spectrometer
# Units: nm, counts
wavelength,intensity
400.0,1250.0
500.0,1800.0
600.0,2100.0
700.0,1600.0"#;

fs::write("../data/spectral_data.csv", metadata_content).unwrap();
println!("Created file with metadata:");
println!("{}", metadata_content);

Created file with metadata:
# Experimental Data


In [16]:
// Load skipping the first 4 metadata lines
let spectral_data = ArrayF64::load_skip("../data/spectral_data.csv", 4).unwrap();
println!("\nLoaded data (skipped 4 metadata lines):");
println!("Shape: {:?}", spectral_data.shape());
println!("Data:\n{:?}", spectral_data);

// The header line "wavelength,intensity" was automatically detected and skipped

# Date: 2024-01-01
# Instrument: Spectrometer
# Units: nm, counts
wavelength,intensity
400.0,1250.0
500.0,1800.0
600.0,2100.0
700.0,1600.0

Loaded data (skipped 4 metadata lines):
Shape: (4, 2)
Data:
Array { inner: [
[400.0, 1250.0],


In [17]:
// Extract wavelengths and intensities
let wavelengths = (0..spectral_data.nrows())
    .map(|i| spectral_data.get(i, 0).unwrap())
    .collect::<Vec<_>>();
let intensities = (0..spectral_data.nrows())
    .map(|i| spectral_data.get(i, 1).unwrap())
    .collect::<Vec<_>>();

println!("Extracted data:");
for (w, i) in wavelengths.iter().zip(intensities.iter()) {
    println!("  {:.1} nm: {:.1} counts", w, i);
}

[500.0, 1800.0],
[600.0, 2100.0],
[700.0, 1600.0],
] }
Extracted data:
  400.0 nm: 1250.0 counts
  500.0 nm: 1800.0 counts
  600.0 nm: 2100.0 counts
  700.0 nm: 1600.0 counts


()

## 6. Error Handling

The I/O module provides helpful error messages for common issues.

In [18]:
// Try to load a non-existent file
match ArrayF64::load("../data/nonexistent.csv") {
    Ok(_) => println!("Unexpected success"),
    Err(e) => println!("Expected error for missing file: {:?}", e),
}

// Try to load a file with inconsistent dimensions
let bad_content = "1.0,2.0,3.0\n4.0,5.0\n6.0,7.0,8.0,9.0";
fs::write("../data/bad_dimensions.csv", bad_content).unwrap();

match ArrayF64::load("../data/bad_dimensions.csv") {
    Ok(_) => println!("Unexpected success"),
    Err(e) => println!("Expected error for inconsistent dimensions: {:?}", e),
}

Expected error for missing file: InvalidDimensions { rows: 0, cols: 0 }
Expected error for inconsistent dimensions: InvalidDimensions { rows: 3, cols: 2 }


()

## 7. Real-World Example: Student Grades

A practical example showing how to work with real data.

In [19]:
// Create student grades data
let grades_content = r#"# Fall 2024 Final Grades
# Course: Advanced Mathematics
student_id,math,science,english
101,85.5,92.0,78.5
102,90.0,88.5,85.0
103,75.5,95.0,82.0
104,88.0,91.5,89.0
105,92.5,87.0,86.5"#;

fs::write("../data/student_grades.csv", grades_content).unwrap();
println!("Student grades file created:");
println!("{}", grades_content);

Student grades file created:
# Fall 2024 Final Grades
# Course: Advanced Mathematics


In [20]:
// Load the grades (skip 2 comment lines)
let grades = ArrayF64::load_skip("../data/student_grades.csv", 2).unwrap();
println!("\nLoaded grades matrix:");
println!("Shape: {:?} (students × subjects)", grades.shape());
println!("Grades:\n{:?}", grades);

student_id,math,science,english
101,85.5,92.0,78.5
102,90.0,88.5,85.0
103,75.5,95.0,82.0
104,88.0,91.5,89.0
105,92.5,87.0,86.5

Loaded grades matrix:
Shape: (5, 4) (students × subjects)
Grades:
Array { inner: [


In [21]:
// Calculate statistics
let (n_students, n_subjects) = grades.shape();

// Student averages (skip student_id column)
println!("\nStudent averages:");
for i in 0..n_students {
    let sum: f64 = (1..n_subjects).map(|j| grades.get(i, j).unwrap()).sum(); // Skip column 0 (student_id)
    let avg = sum / (n_subjects - 1) as f64; // Divide by 3 subjects, not 4 columns
    println!("  Student {}: {:.1}", i + 1, avg);
}

// Subject averages (skip student_id column)
let subjects = ["Student ID", "Math", "Science", "English"];
println!("\nSubject averages:");
for j in 1..n_subjects { // Start from 1 to skip student_id column
    let sum: f64 = (0..n_students).map(|i| grades.get(i, j).unwrap()).sum();
    let avg = sum / n_students as f64;
    println!("  {}: {:.1}", subjects[j], avg);
}

[101.0, 85.5, 92.0, 78.5],
[102.0, 90.0, 88.5, 85.0],
[103.0, 75.5, 95.0, 82.0],
[104.0, 88.0, 91.5, 89.0],
[105.0, 92.5, 87.0, 86.5],
] }

Student averages:
  Student 1: 85.3
  Student 2: 87.8
  Student 3: 84.2
  Student 4: 89.5
  Student 5: 88.7

Subject averages:
  Math: 86.3
  Science: 90.8
  English: 84.2


()

In [22]:
// Save processed results with high precision
let processed_grades = grades.clone(); // In real code, you'd apply transformations
processed_grades.save_with_precision("../data/processed_grades.csv", 1).unwrap();
println!("✓ Processed grades saved with 1 decimal precision");

let saved_content = fs::read_to_string("../data/processed_grades.csv").unwrap();
println!("\nSaved processed grades:");
println!("{}", saved_content);

✓ Processed grades saved with 1 decimal precision

Saved processed grades:
101.0,85.5,92.0,78.5
102.0,90.0,88.5,85.0
103.0,75.5,95.0,82.0
104.0,88.0,91.5,89.0
105.0,92.5,87.0,86.5



## Summary

The RustLab I/O module provides a clean, math-first API with just 2 core functions:

### For Saving:
- `data.save("file.csv")` - Basic save with default precision
- `data.save_with_precision("file.csv", 10)` - High precision save

### For Loading:
- `ArrayF64::load("file.csv")` - Basic load
- `ArrayF64::load_skip("file.csv", 3)` - Load skipping metadata rows

### Key Features:
- ✅ **Automatic format detection** (CSV vs text)
- ✅ **Smart header detection** (skips non-numeric headers)
- ✅ **Scientific notation** for extreme values
- ✅ **High precision support** (1-15+ digits)
- ✅ **Works with both** matrices and vectors
- ✅ **NumPy-equivalent functionality**
- ✅ **Clean error handling**

This provides all the I/O functionality you need for scientific computing with a simple, intuitive API!